In [2]:
import yaml

In [3]:
# load menu
with open("../mnt/city-directories/01-user-input/menu.yml", 'r') as f:
    menu = yaml.safe_load(f)

In [4]:
if menu['toolbox']:
    import os
    import glob
    import math
    import geopandas as gpd
    import pandas as pd
    import numpy as np
    import pint
    from pathlib import Path
    import matplotlib.pyplot as plt
    import rasterio
    from rasterio.warp import calculate_default_transform, reproject, Resampling
    from rasterio.plot import show
    from rasterstats import zonal_stats
    from osgeo import gdal, gdalconst
    from scipy.ndimage import generic_filter
    from shapely.geometry import LineString
    from shapely.ops import linemerge, unary_union
    import fiona
    import osmnx as ox
    from shapely.geometry import LineString, mapping
    from skimage import measure
    from shapely.ops import unary_union
    from rasterstats import zonal_stats
    from affine import Affine
    from rasterio.features import geometry_mask
    import fiona
    from rasterio.crs import CRS
    import warnings
    from rasterio.merge import merge 
    from rasterio.transform import from_bounds
    import csv
    from shapely.geometry import LineString, MultiPoint
    from shapely.ops import split, snap
    from rasterio.mask import mask
    import rasterio.features
    from rasterio.enums import Resampling
    from rasterio.vrt import WarpedVRT
    from rasterio.features import shapes
    from shapely.geometry import shape
    import rasterio.warp

In [6]:
# SET UP ##############################################

# load city inputs files, to be updated for each city scan
with open("../mnt/city-directories/01-user-input/city_inputs.yml", 'r') as f:
    city_inputs = yaml.safe_load(f)

city = city_inputs['city_name'].replace(' ', '_').lower()
country = city_inputs['country_name'].replace(' ', '_').lower()
# load global inputs, such as data sources that generally remain the same across scans

with open("../global_inputs.yml", 'r') as f:
    global_inputs = yaml.safe_load(f)



# transform the input shp to correct prj (epsg 4326)
aoi_file = gpd.read_file(os.path.join('..', city_inputs['AOI_path'])).to_crs(epsg=4326)
features = aoi_file.geometry

# Define output folder ---------
output_folder = Path('../mnt/city-directories/02-process-output')
output_folder = Path('../mnt/city-directories/02-process-output')

if not os.path.exists(output_folder):
    os.mkdir(output_folder)
    


In [7]:
#merge pluvial 

def merge_pluvial_files():
    matching_files = glob.glob(os.path.join(output_folder, f"{city}_pluvial_2020_*.tif"))
    
    if matching_files:
        src_files_to_merge = [rasterio.open(pluvial_file) for pluvial_file in matching_files]
        
        try:
            merged_data, merged_transform = merge(src_files_to_merge)
            merged_crs = src_files_to_merge[0].crs
            
            output_file = os.path.join(output_folder, f"{city}_merged_pluvial_data.tif")
            with rasterio.open(output_file, 'w', driver='GTiff',
                               width=merged_data.shape[2], height=merged_data.shape[1],
                               count=1, dtype=merged_data.dtype,
                               crs=merged_crs, transform=merged_transform) as dst:
                dst.write(merged_data)
    
            print(f"Merged pluvial data saved as {output_file}")
            return output_file
        except Exception as e:
            print(f"Error occurred while merging: {e}")
            return None
    else:
        print("Error: No pluvial files found.")
        return None
merge_pluvial_files()




Merged pluvial data saved as ../mnt/city-directories/02-process-output/guiyang_merged_pluvial_data.tif


'../mnt/city-directories/02-process-output/guiyang_merged_pluvial_data.tif'

In [8]:
# #merge pluvial 
# def merge_pluvial_files_UTM():
#     matching_files = glob.glob(os.path.join(output_folder, f"{city}_pluvial_2020_*_utm.tif"))
#     matching_files=
    
#     if matching_files:
#         src_files_to_merge = [rasterio.open(pluvial_file) for pluvial_file in matching_files]
        
#         merged_data, merged_transform = merge(src_files_to_merge)
#         merged_crs = src_files_to_merge[0].crs
        
#         output_file = os.path.join(output_folder, f"{city}_merged_pluvial_data_utm.tif")
#         with rasterio.open(output_file, 'w', driver='GTiff',
#                            width=merged_data.shape[2], height=merged_data.shape[1],
#                            count=1, dtype=merged_data.dtype,
#                            crs=merged_crs, transform=merged_transform) as dst:
#             dst.write(merged_data)

#         print(f"Merged pluvial data saved as {output_file}")
#         return output_file
#     else:
#         print("Error: No pluvial files found.")
#         return None
# merge_pluvial_files_UTM()

In [9]:
def merge_fluvial_files():
    matching_files = glob.glob(os.path.join(output_folder, f"{city}_fluvial_2020_*.tif"))
    
    if matching_files:
        src_files_to_merge = [rasterio.open(fluvial_file) for fluvial_file in matching_files]
        
        merged_data, merged_transform = merge(src_files_to_merge)
        merged_crs = src_files_to_merge[0].crs
        
        output_file = os.path.join(output_folder, f"{city}_merged_fluvial_data.tif")
        with rasterio.open(output_file, 'w', driver='GTiff',
                           width=merged_data.shape[2], height=merged_data.shape[1],
                           count=1, dtype=merged_data.dtype,
                           crs=merged_crs, transform=merged_transform) as dst:
            dst.write(merged_data)

        print(f"Merged fluvial data saved as {output_file}")
        return output_file
    else:
        print("Error: No fluvial files found.")
        return None
merge_fluvial_files()

Merged fluvial data saved as ../mnt/city-directories/02-process-output/guiyang_merged_fluvial_data.tif


'../mnt/city-directories/02-process-output/guiyang_merged_fluvial_data.tif'

In [10]:
def merge_fluvial_files_UTM():
    matching_files = glob.glob(os.path.join(output_folder, f"{city}_fluvial_2020_utm.tif"))
    
    if matching_files:
        src_files_to_merge = [rasterio.open(fluvial_file) for fluvial_file in matching_files]
        
        merged_data, merged_transform = merge(src_files_to_merge)
        merged_crs = src_files_to_merge[0].crs  # Use the CRS of the first file

        output_file = os.path.join(output_folder, f"{city}_merged_fluvial_data_utm.tif")
        with rasterio.open(output_file, 'w', driver='GTiff',
                           width=merged_data.shape[2], height=merged_data.shape[1],
                           count=1, dtype=merged_data.dtype,
                           crs=merged_crs, transform=merged_transform) as dst:
            dst.write(merged_data)

        print(f"Merged fluvial data saved as {output_file}")
        return output_file
    else:
        print("Error: No fluvial files found.")
        return None
merge_fluvial_files_UTM()

Error: No fluvial files found.


In [11]:
def merge_comb_files_UTM():
    matching_files = glob.glob(os.path.join(output_folder, f"{city}_comb_2020_utm.tif"))
    
    if matching_files:
        src_files_to_merge = [rasterio.open(comb_file) for comb_file in matching_files]
        
        merged_data, merged_transform = merge(src_files_to_merge)
        merged_crs = src_files_to_merge[0].crs  # Use the CRS of the first file

        output_file = os.path.join(output_folder, f"{city}_merged_comb_data_utm.tif")
        with rasterio.open(output_file, 'w', driver='GTiff',
                           width=merged_data.shape[2], height=merged_data.shape[1],
                           count=1, dtype=merged_data.dtype,
                           crs=merged_crs, transform=merged_transform) as dst:
            dst.write(merged_data)

        print(f"Merged combined data saved as {output_file}")
        return output_file
    else:
        print("Error: No combined files found.")
        return None
merge_comb_files_UTM()

Merged combined data saved as ../mnt/city-directories/02-process-output/guiyang_merged_comb_data_utm.tif


'../mnt/city-directories/02-process-output/guiyang_merged_comb_data_utm.tif'

In [28]:
#merge coastal files and save a merged file  (everything with a value is 1, otherwise 0 )

In [12]:
#resampling data
def resample_raster(input_raster, target_shape):
    # Resample raster to match the target shape
    data = input_raster.read(1, out_shape=target_shape, resampling=Resampling.nearest)
    return data

In [15]:

#PLOT CHECK
def plot_data():
    if menu.get('flood') and menu.get('wsf'):
        pluvial_path = os.path.join(output_folder, f"{city}_merged_pluvial_data_utm.tif")
        wsf_path = os.path.join(output_folder, f"{city}_wsf_utm.tif")

        if features.empty:
            print("No features to plot.")
            return

        if features.crs is None:
            print("Features do not have a CRS defined.")
            return

        with rasterio.open(pluvial_path) as pluvial_src:
            pluvial_crs = pluvial_src.crs

        features_utm = features.to_crs(pluvial_crs)

        
        fig, ax = plt.subplots(figsize=(10, 10))
        features_utm.plot(ax=ax, facecolor='none', edgecolor='red')


        with rasterio.open(pluvial_path) as pluvial_src:
            show(pluvial_src, ax=ax, cmap='Blues', alpha=0.5)


        with rasterio.open(wsf_path) as wsf_src:
            show(wsf_src, ax=ax, cmap='Reds', alpha=0.5)

        ax.set_title("Features, Pluvial Data, and WSF")
        ax.set_xlabel("Longitude")
        ax.set_ylabel("Latitude")
        plt.show()
        plt.savefig('plot.png')  
    else:
        print("Flood or WSF menu not selected.")




In [31]:
#WSF and Pu

def get_pu_wsf():
    if menu.get('flood') and menu.get('wsf'):
        pu_path = os.path.join(output_folder, f"{city}_merged_pluvial_data_utm.tif")
        wsf_path = os.path.join(output_folder, f"{city}_wsf_utm.tif")


        with rasterio.open(pu_path) as pluvial_src:
            pluvial_crs = pluvial_src.crs


        features_utm = features.to_crs(pluvial_crs)


        with rasterio.open(pu_path) as src_pluvial:
            pluvial_data, pluvial_transform = mask(src_pluvial, features_utm.geometry, crop=True)
            pluvial_data = pluvial_data[0]  
            pluvial_affine = pluvial_transform
            pluvial_resolution = abs(pluvial_transform[0] * pluvial_transform[4])  

        with rasterio.open(wsf_path) as src_wsf:
            wsf_data, wsf_transform = mask(src_wsf, features_utm.geometry, crop=True)
            wsf_data = wsf_data[0]  
            wsf_affine = wsf_transform

        
        min_height = min(pluvial_data.shape[0], wsf_data.shape[0])
        min_width = min(pluvial_data.shape[1], wsf_data.shape[1])
        pluvial_data = pluvial_data[:min_height, :min_width]
        wsf_data = wsf_data[:min_height, :min_width]

        
        unique_years = np.unique(wsf_data)
        unique_years = unique_years[unique_years != 0]
        unique_years = unique_years[unique_years != 1984]  

        stats_by_year = {}

        for year in unique_years:
            masked_wsf_data = np.where(wsf_data == year, 1, 0)
            masked_flooded_data = masked_wsf_data * pluvial_data
            stats = zonal_stats(features_utm.geometry, masked_flooded_data, affine=pluvial_transform, stats="sum", nodata=-9999)
            area = stats[0]['sum'] * pluvial_resolution  

            stats_by_year[year] = area/1000000

        return stats_by_year

    else:
        print("Flood or WSF menu not selected.")
        return None


In [32]:
#Fluvial and WSF

def get_fu_wsf():
    if menu.get('flood') and menu.get('wsf'):
        fu_path = os.path.join(output_folder, f"{city}_merged_fluvial_data_utm.tif")  
        wsf_path = os.path.join(output_folder, f"{city}_wsf_utm.tif")


        with rasterio.open(fu_path) as src_fluvial:
            fluvial_crs = src_fluvial.crs

  
        features_utm = features.to_crs(fluvial_crs)

        with rasterio.open(fu_path) as src_fluvial:  
            fluvial_data, fluvial_transform = mask(src_fluvial, features_utm.geometry, crop=True)
            fluvial_data = fluvial_data[0]  
            fluvial_affine = fluvial_transform
            fluvial_resolution = abs(fluvial_transform[0] * fluvial_transform[4])  
            print(fluvial_resolution)

        with rasterio.open(wsf_path) as src_wsf:
            wsf_data, wsf_transform = mask(src_wsf, features_utm.geometry, crop=True)
            wsf_data = wsf_data[0]  
            wsf_affine = wsf_transform


        min_height = min(fluvial_data.shape[0], wsf_data.shape[0])
        min_width = min(fluvial_data.shape[1], wsf_data.shape[1])
        fluvial_data = fluvial_data[:min_height, :min_width]
        wsf_data = wsf_data[:min_height, :min_width]

        min_height = min(fluvial_data.shape[0], wsf_data.shape[0])
        min_width = min(fluvial_data.shape[1], wsf_data.shape[1])
        fluvial_data = fluvial_data[:min_height, :min_width]
        wsf_data = wsf_data[:min_height, :min_width]


        unique_years = np.unique(wsf_data)
        unique_years = unique_years[unique_years != 0]
        unique_years = unique_years[unique_years != 1984]  

        stats_by_year = {}
        stats_by_year = {}

        for year in unique_years:
            masked_wsf_data = np.where(wsf_data == year, 1, 0)
            masked_flooded_data = masked_wsf_data * fluvial_data  
            stats = zonal_stats(features_utm.geometry, masked_flooded_data, affine=fluvial_transform, stats="sum", nodata=-9999)
            area = stats[0]['sum'] * fluvial_resolution  
        for year in unique_years:
            masked_wsf_data = np.where(wsf_data == year, 1, 0)
            masked_flooded_data = masked_wsf_data * fluvial_data  
            stats = zonal_stats(features_utm.geometry, masked_flooded_data, affine=fluvial_transform, stats="sum", nodata=-9999)
            area = stats[0]['sum'] * fluvial_resolution  

            stats_by_year[year] = area/1000000

        return stats_by_year
        return stats_by_year

    else:
        print("Flood or WSF menu not selected.")
        return None

In [33]:
#Combined and WSF

def get_comb_wsf():
    if menu.get('flood') and menu.get('wsf'):
        comb_path = os.path.join(output_folder, f"{city}_comb_2020_utm.tif")  
        wsf_path = os.path.join(output_folder, f"{city}_wsf_utm.tif")


        with rasterio.open(comb_path) as src_comb:
            comb_crs = src_comb.crs


        features_utm = features.to_crs(comb_crs)

        with rasterio.open(comb_path) as src_comb:  
            comb_data, comb_transform = mask(src_comb, features_utm.geometry, crop=True)
            comb_data = comb_data[0]  
            comb_data = np.ma.masked_where(comb_data == 65535, comb_data)
            comb_affine = comb_transform
            comb_resolution = abs(comb_transform[0] * comb_transform[4])

        with rasterio.open(wsf_path) as src_wsf:
            wsf_data, wsf_transform = mask(src_wsf, features_utm.geometry, crop=True)
            wsf_data = wsf_data[0]  
            wsf_affine = wsf_transform


        min_height = min(comb_data.shape[0], wsf_data.shape[0])
        min_width = min(comb_data.shape[1], wsf_data.shape[1])
        comb_data = comb_data[:min_height, :min_width]
        wsf_data = wsf_data[:min_height, :min_width]


        unique_years = np.unique(wsf_data)
        unique_years = unique_years[unique_years != 0]
        unique_years = unique_years[unique_years != 1984]  

        stats_by_year = {}

        for year in unique_years:
            masked_wsf_data = np.where(wsf_data == year, 1, 0)
            masked_flooded_data = masked_wsf_data * comb_data  
            stats = zonal_stats(features_utm.geometry, masked_flooded_data, affine=comb_transform, stats="sum", nodata=-9999)
            area = stats[0]['sum'] * comb_resolution  

            stats_by_year[year] = area/4000000

        return stats_by_year

    else:
        print("Flood or WSF menu not selected.")
        return None

In [34]:
#Do the same for coastal files 

In [47]:
def get_pu_pop_norm():
    with open("../global_inputs.yml", 'r') as f:
        global_inputs = yaml.safe_load(f)
        
    if menu.get('flood') and menu.get('population'):
        pop_path = os.path.join(output_folder, f"{city}_population.tif")
        try:
            
            with rasterio.open(pop_path) as pop_src:
                pop_shape = pop_src.shape
                pop_transform = pop_src.transform
                
                
                pu_path = os.path.join(output_folder, f"{city}_merged_pluvial_data.tif")
                with rasterio.open(pu_path) as pu_src:
                    pu_data = pu_src.read(1)
                    pu_resampled = np.empty(pop_shape, dtype=pu_data.dtype)
                    rasterio.warp.reproject(
                        source=pu_data,
                        destination=pu_resampled,
                        src_transform=pu_src.transform,
                        src_crs=pu_src.crs,
                        dst_transform=pop_transform,
                        dst_crs=pop_src.crs,
                        resampling=rasterio.enums.Resampling.nearest
                    )


                pop_data = pop_src.read(1)

                pop_data_clipped = np.clip(pop_data, 0, None)
                

                non_zero_pop_data = pop_data_clipped[pop_data_clipped != 0]
                

                pop_percentile_60 = np.percentile(non_zero_pop_data, 60)
                print("60th Percentile of Population Data (excluding zeros):", pop_percentile_60)

                total_count = np.sum((pu_resampled >= 1) & (pop_data_clipped > pop_percentile_60))
                total_pixels = np.sum(pop_data_clipped > 0)

                percentage = (total_count / total_pixels) * 100
                print(f"{percentage:.2f}% of densely populated areas are located within the rainwater flood risk zone with a minimum depth of 15 cm")

                
                csv_path = os.path.join(output_folder, 'pu_pop_area.csv')
                df = pd.DataFrame({'File Name': 'Combined', 'Percentage': [percentage]})
                df.to_csv(csv_path, index=False)
                print(f"Result saved to {csv_path}")
                
        except Exception as e:
            print(f"Error opening or processing raster: {e}")
            return
    else:
        print("Flood or population menu not selected.")


60th Percentile of Population Data (excluding zeros): 39.820377349853516
6.66% of densely populated areas are located within the rainwater flood risk zone with a minimum depth of 15 cm
Result saved to mnt/city-directories/02-process-output/pu_pop_area.csv


In [46]:
def get_fu_pop_norm():
    with open("../global_inputs.yml", 'r') as f:
        global_inputs = yaml.safe_load(f)
        
    if menu.get('flood') and menu.get('population'):
        pop_path = os.path.join(output_folder, f"{city}_population.tif")
        try:

            with rasterio.open(pop_path) as pop_src:
                pop_shape = pop_src.shape
                pop_transform = pop_src.transform
                

                fu_path = os.path.join(output_folder, f"{city}_fluvial_2020_gt10.tif")
                with rasterio.open(fu_path) as fu_src:
                    fu_data = fu_src.read(1)
                    fu_resampled = np.empty(pop_shape, dtype=fu_data.dtype)
                    rasterio.warp.reproject(
                        source=fu_data,
                        destination=fu_resampled,
                        src_transform=fu_src.transform,
                        src_crs=fu_src.crs,
                        dst_transform=pop_transform,
                        dst_crs=pop_src.crs,
                        resampling=rasterio.enums.Resampling.nearest
                    )

              
                pop_data = pop_src.read(1)

                pop_data_clipped = np.clip(pop_data, 0, None)
                
              
                non_zero_pop_data = pop_data_clipped[pop_data_clipped != 0]
                
                
                pop_percentile_60 = np.percentile(non_zero_pop_data, 60)
                print("60th Percentile of Population Data (excluding zeros):", pop_percentile_60)

                
                total_count = np.sum((fu_resampled >= 1) & (pop_data_clipped > pop_percentile_60))
                total_pixels = np.sum(pop_data_clipped > 0)

                percentage = (total_count / total_pixels) * 100
                print(f"{percentage:.2f}% of densely populated areas are located within the fluvial flood risk zone with a minimum depth of 15 cm")

                
                csv_path = os.path.join(output_folder, 'fu_pop_area.csv')
                df = pd.DataFrame({'File Name': 'Combined', 'Percentage': [percentage]})
                df.to_csv(csv_path, index=False)
                print(f"Result saved to {csv_path}")
                
        except Exception as e:
            print(f"Error opening or processing raster: {e}")
            return
    else:
        print("Flood or population menu not selected.")


60th Percentile of Population Data (excluding zeros): 39.820377349853516
0.16% of densely populated areas are located within the fluvial flood risk zone with a minimum depth of 15 cm
Result saved to mnt/city-directories/02-process-output/fu_pop_area.csv


In [45]:
def get_comb_pop_norm():
    with open("../global_inputs.yml", 'r') as f:
        global_inputs = yaml.safe_load(f)
        
    if menu.get('flood') and menu.get('population'):
        pop_path = os.path.join(output_folder, f"{city}_population.tif")
        try:
            
            with rasterio.open(pop_path) as pop_src:
                pop_shape = pop_src.shape
                pop_transform = pop_src.transform
                
                comb_path = os.path.join(output_folder, f"{city}_comb_2020.tif")
                with rasterio.open(comb_path) as comb_src:
                    comb_data = comb_src.read(1)
                    comb_resampled = np.empty(pop_shape, dtype=comb_data.dtype)
                    rasterio.warp.reproject(
                        source=comb_data,
                        destination=comb_resampled,
                        src_transform=comb_src.transform,
                        src_crs=comb_src.crs,
                        dst_transform=pop_transform,
                        dst_crs=pop_src.crs,
                        resampling=rasterio.enums.Resampling.nearest
                    )

                pop_data = pop_src.read(1)

                pop_data_clipped = np.clip(pop_data, 0, None)
                
                non_zero_pop_data = pop_data_clipped[pop_data_clipped != 0]
                
                pop_percentile_60 = np.percentile(non_zero_pop_data, 60)
                print("60th Percentile of Population Data (excluding zeros):", pop_percentile_60)

                total_count = np.sum((comb_resampled >= 1) & (pop_data_clipped > pop_percentile_60))
                total_pixels = np.sum(pop_data_clipped > 0)

                percentage = (total_count / total_pixels) * 100
                print(f"{percentage:.2f}% of densely populated areas are located within the combined flood risk zone with a minimum depth of 15 cm")

                csv_path = os.path.join(output_folder, 'comb_pop_area.csv')
                df = pd.DataFrame({'File Name': 'Combined', 'Percentage': [percentage]})
                df.to_csv(csv_path, index=False)
                print(f"Result saved to {csv_path}")
                
        except Exception as e:
            print(f"Error opening or processing raster: {e}")
            return
    else:
        print("Flood or population menu not selected.")

60th Percentile of Population Data (excluding zeros): 39.820377349853516
40.00% of densely populated areas are located within the combined flood risk zone with a minimum depth of 15 cm
Result saved to mnt/city-directories/02-process-output/comb_pop_area.csv


In [17]:
# Pu Amenities

def get_pu_am():
    with open("../global_inputs.yml", 'r') as f:
        global_inputs = yaml.safe_load(f)
        
    if menu.get('flood') and menu.get('amenities'):  
        pu_path = os.path.join(output_folder, f"{city}_pluvial_2020_lt1.tif")
        try:
            with rasterio.open(pu_path) as pu_src:
                merged_pluvial_data = pu_src.read(1)
                merged_pluvial_data_transform = pu_src.transform
                merged_pluvial_data_shape = merged_pluvial_data.shape
        except Exception as e:
            print(f"Error opening merged pluvial data raster: {e}")
            return

        
        stats_list = []
        for category in ['health', 'police', 'fire','schools']:
            shapefile_path = os.path.join(output_folder, f"{city}_osm_{category}", f"{city}_osm_{category}.shp")
            try:
                amenities = gpd.read_file(shapefile_path)

                with rasterio.open(pu_path) as src:
                    affine = src.transform

                stats = zonal_stats(amenities, merged_pluvial_data, nodata=0, affine=affine, stats=["count"], geojson_out=True)

                count_overlap = sum([feature["properties"]["count"] for feature in stats])
                total_count = len(amenities)
                percentage = (count_overlap / total_count) * 100

                stats_list.append({'Category': category, 'Overlap': count_overlap, 'Total': total_count, 'Percentage': percentage})

                print(f"{count_overlap} of {total_count} ({percentage:.2f}%) {category} are located in a riverine flood risk zone with a minimum depth of 15 cm.")

            except Exception as e:
                if category == 'fire':
                    print("Fire stations do not exist")
                else:
                    print(f"Error processing {category} shapefile: {e}")
        
        df = pd.DataFrame(stats_list)
        
        
        excel_file = os.path.join(output_folder, 'pu_osmpt.xlsx')
        df.to_excel(excel_file, index=False)
        print(f"Statistics saved to {excel_file}")

# Suppress warnings
warnings.filterwarnings("ignore", category=rasterio.errors.NotGeoreferencedWarning)
warnings.filterwarnings("ignore", category=RuntimeWarning)




In [18]:
#PU roads
def get_pu_roads():
    with open("../global_inputs.yml", 'r') as f:
        global_inputs = yaml.safe_load(f)

    pu_path = os.path.join(output_folder, f"{city}_pluvial_2020_lt1.tif")
    roads_path = os.path.join(f'mnt/city-directories/02-process-output/{city}_road_network/{city}_edges.shp')

    try:
        with rasterio.open(pu_path) as pu_src:
            merged_pluvial_data = pu_src.read(1)
            transform = pu_src.transform  
            nodata_value = pu_src.nodata  

            mask = (merged_pluvial_data != nodata_value) & (merged_pluvial_data != 0)
            merged_pluvial_data_masked = np.ma.masked_array(merged_pluvial_data, mask=mask)

            shapes_gen = shapes(merged_pluvial_data_masked, transform=transform)
            merged_pluvial_polygons = [shape(shape_item) for shape_item, _ in shapes_gen]
            pluvial_geometry = gpd.GeoDataFrame(geometry=merged_pluvial_polygons, crs=pu_src.crs)

    except Exception as e:
        print(f"Error opening merged pluvial data raster: {e}")
        return

    try:
        roads = gpd.read_file(roads_path)
    except Exception as e:
        print(f"Error reading road network shapefile: {e}")
        return
    roads = roads.to_crs(pluvial_geometry.crs)

    highways_filtered = roads[(roads["highway"] == 'primary') | 
                              (roads["highway"] == 'trunk') | 
                              (roads["highway"] == 'motorway')]

    exploded_highways = highways_filtered.explode(index_parts=False)
    
    intersections = gpd.overlay(exploded_highways, pluvial_geometry, how='intersection')

    intersections['length'] = intersections.length
    
    total_length = intersections['length'].sum()
    
    total_length_highways = exploded_highways['length'].sum()

    percentage = (total_length / total_length_highways) * 10000
    
    print(f"Total length of highways intersecting pluvial data: {total_length:.2f} km")
    print(f"Percentage of highways intersecting pluvial data: {percentage:.4f}%")




In [19]:
# Fu Amenities

def get_fu_am():
    with open("../global_inputs.yml", 'r') as f:
        global_inputs = yaml.safe_load(f)
        
    if menu.get('flood') and menu.get('amenities'):  
        fu_path = os.path.join(output_folder, f"{city}_fluvial_2020_lt1.tif")
        try:
            with rasterio.open(fu_path) as fu_src:
                merged_fluvial_data = fu_src.read(1)
                merged_fluvial_transform = fu_src.transform
                merged_fluvial_shape = merged_fluvial_data.shape
        except Exception as e:
            print(f"Error opening merged fluvial data raster: {e}")
            return

        
        stats_list = []
        for category in ['health', 'police', 'fire','schools']:
            shapefile_path = os.path.join(output_folder, f"{city}_osm_{category}", f"{city}_osm_{category}.shp")
            try:
                amenities = gpd.read_file(shapefile_path)

                with rasterio.open(fu_path) as src:
                    affine = src.transform

                stats = zonal_stats(amenities, merged_fluvial_data, nodata=0, affine=affine, stats=["count"], geojson_out=True)

                count_overlap = sum([feature["properties"]["count"] for feature in stats])
                total_count = len(amenities)
                percentage = (count_overlap / total_count) * 100

                stats_list.append({'Category': category, 'Overlap': count_overlap, 'Total': total_count, 'Percentage': percentage})

                print(f"{count_overlap} of {total_count} ({percentage:.2f}%) {category} are located in a riverine flood risk zone with a minimum depth of 15 cm.")

            except Exception as e:
                if category == 'fire':
                    print("Fire stations do not exist")
                else:
                    print(f"Error processing {category} shapefile: {e}")
        
        df = pd.DataFrame(stats_list)
        
        
        excel_file = os.path.join(output_folder, 'fu_osmpt.xlsx')
        df.to_excel(excel_file, index=False)
        print(f"Statistics saved to {excel_file}")

# Suppress warnings
warnings.filterwarnings("ignore", category=rasterio.errors.NotGeoreferencedWarning)
warnings.filterwarnings("ignore", category=RuntimeWarning)


# Fu Amenities

def get_fu_am():
    with open("../global_inputs.yml", 'r') as f:
        global_inputs = yaml.safe_load(f)
        
    if menu.get('flood') and menu.get('amenities'):  
        fu_path = os.path.join(output_folder, f"{city}_fluvial_2020_lt1.tif")
        try:
            with rasterio.open(fu_path) as fu_src:
                merged_fluvial_data = fu_src.read(1)
                merged_fluvial_transform = fu_src.transform
                merged_fluvial_shape = merged_fluvial_data.shape
        except Exception as e:
            print(f"Error opening merged fluvial data raster: {e}")
            return

        
        stats_list = []
        for category in ['health', 'police', 'fire','schools']:
            shapefile_path = os.path.join(output_folder, f"{city}_osm_{category}", f"{city}_osm_{category}.shp")
            try:
                amenities = gpd.read_file(shapefile_path)

                with rasterio.open(fu_path) as src:
                    affine = src.transform

                stats = zonal_stats(amenities, merged_fluvial_data, nodata=0, affine=affine, stats=["count"], geojson_out=True)

                count_overlap = sum([feature["properties"]["count"] for feature in stats])
                total_count = len(amenities)
                percentage = (count_overlap / total_count) * 100

                stats_list.append({'Category': category, 'Overlap': count_overlap, 'Total': total_count, 'Percentage': percentage})

                print(f"{count_overlap} of {total_count} ({percentage:.2f}%) {category} are located in a riverine flood risk zone with a minimum depth of 15 cm.")

            except Exception as e:
                if category == 'fire':
                    print("Fire stations do not exist")
                else:
                    print(f"Error processing {category} shapefile: {e}")
        
        df = pd.DataFrame(stats_list)
        
        
        excel_file = os.path.join(output_folder, 'fu_osmpt.xlsx')
        df.to_excel(excel_file, index=False)
        print(f"Statistics saved to {excel_file}")

# Suppress warnings
warnings.filterwarnings("ignore", category=rasterio.errors.NotGeoreferencedWarning)
warnings.filterwarnings("ignore", category=RuntimeWarning)




In [189]:

def get_fu_roads():
    with open("../global_inputs.yml", 'r') as f:
        global_inputs = yaml.safe_load(f)
    fu_path = os.path.join(output_folder, f"{city}_fluvial_2020_lt1.tif")
    roads_path = os.path.join(f'mnt/city-directories/02-process-output/{city}_road_network/{city}_edges.shp')

    try:
        with rasterio.open(fu_path) as fu_src:
            fu_data = fu_src.read(1)
            transform = fu_src.transform  
            nodata_value = fu_src.nodata  

            mask = (fu_data != nodata_value) & (fu_data != 0)
            fu_data_masked = np.ma.masked_array(fu_data, mask=mask)

            shapes_gen = shapes(fu_data_masked, transform=transform)
            fu_polygons = [shape(shape_item) for shape_item, _ in shapes_gen]
            fu_geometry = gpd.GeoDataFrame(geometry=fu_polygons, crs=fu_src.crs)

    except Exception as e:
        print(f"Error opening fu data raster: {e}")
        return

    try:
        roads = gpd.read_file(roads_path)
    except Exception as e:
        print(f"Error reading road network shapefile: {e}")
        return
    roads = roads.to_crs(fu_geometry.crs)

    highways_filtered = roads[(roads["highway"] == 'primary') | 
                              (roads["highway"] == 'trunk') | 
                              (roads["highway"] == 'motorway')]

    intersections = gpd.overlay(highways_filtered, fu_geometry, how='intersection')

    intersections['length'] = intersections.length
    
    total_length = intersections['length'].sum()
    
    total_length_fu = highways_filtered['length'].sum()

    percentage = (total_length / total_length_fu) *10000
    
    print(f"Total length of roads intersecting fluvial data: {total_length:.2f} km")
    print(f"Percentage of roads intersecting fluvial data: {percentage:.4f}%")



In [194]:
#Comb Amenities
def get_comb_am():
    with open("../global_inputs.yml", 'r') as f:
        global_inputs = yaml.safe_load(f)
        
    if menu.get('flood') and menu.get('amenities'):  
        comb_path = os.path.join(output_folder, f"{city}_comb_2020.tif")
        try:
            with rasterio.open(comb_path) as comb_src:
                merged_comb_data = comb_src.read(1)
                merged_comb_data = np.ma.masked_where(merged_comb_data == 65535, merged_comb_data)
                merged_comb_data = np.ma.masked_where(merged_comb_data == 0, merged_comb_data)
                merged_comb_transform = comb_src.transform
                merged_comb_shape = merged_comb_data.shape
        except Exception as e:
            print(f"Error opening merged combined data raster: {e}")
            return

        
        stats_list = []
        for category in ['health', 'police', 'fire','schools']:
            shapefile_path = os.path.join(output_folder, f"{city}_osm_{category}", f"{city}_osm_{category}.shp")
            try:
                amenities = gpd.read_file(shapefile_path)

                with rasterio.open(comb_path) as src:
                    affine = src.transform

                stats = zonal_stats(amenities, merged_comb_data, nodata=0, affine=affine, stats=["count"], geojson_out=True)

                count_overlap = sum([feature["properties"]["count"] for feature in stats])
                total_count = len(amenities)
                percentage = (count_overlap / total_count) * 100

                stats_list.append({'Category': category, 'Overlap': count_overlap, 'Total': total_count, 'Percentage': percentage})

                print(f"{count_overlap} of {total_count} ({percentage:.2f}%) {category} are located in a combined flood risk zone with a minimum depth of 15 cm.")

            except Exception as e:
                if category == 'fire':
                    print("Fire stations do not exist")
                else:
                    print(f"Error processing {category} shapefile: {e}")
        
        df = pd.DataFrame(stats_list)
        
        
        excel_file = os.path.join(output_folder, 'comb_osmpt.xlsx')
        df.to_excel(excel_file, index=False)
        print(f"Statistics saved to {excel_file}")
# Suppress warnings
warnings.filterwarnings("ignore", category=rasterio.errors.NotGeoreferencedWarning)
warnings.filterwarnings("ignore", category=RuntimeWarning)

26 of 26 (100.00%) health are located in a combined flood risk zone with a minimum depth of 15 cm.
6 of 6 (100.00%) police are located in a combined flood risk zone with a minimum depth of 15 cm.
2 of 2 (100.00%) fire are located in a combined flood risk zone with a minimum depth of 15 cm.
138 of 138 (100.00%) schools are located in a combined flood risk zone with a minimum depth of 15 cm.
Statistics saved to mnt/city-directories/02-process-output/comb_osmpt.xlsx


In [35]:
#Coastal WSF

In [36]:
#Coastal Population

In [37]:
#Coastal Amenities